In [1]:
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.0/901.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-04 07:17:28.765819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764832648.951684      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764832649.005302      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


# 蒸馏训练
## 1. 蒸馏模型 Trainer

In [3]:
# 计算反向kl散度
def compute_rkl(logits, teacher_logits, target, padding_id, reduction="sum", temp=1.0):
    logits = logits / temp
    teacher_logits = teacher_logits / temp
    probs = torch.softmax(logits, -1, dtype=torch.float32)
    log_probs = torch.log_softmax(logits, -1, dtype=torch.float32)
    teacher_log_probs = torch.log_softmax(teacher_logits, -1,dtype=torch.float32)
    kl = (probs * (log_probs - teacher_log_probs))
    kl = kl.sum(-1)
    
    if reduction == "sum":
        pad_mask = target.eq(padding_id)
        kl = kl.masked_fill_(pad_mask, 0.0)
        kl = kl.sum()
    return kl


In [4]:
class KDTrainer(SFTTrainer):
    def __init__(self, *args, teacher_model=None, if_use_entropy=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.if_use_entropy = if_use_entropy
        
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs_student = model(**inputs)
        
        with torch.no_grad():
            teacher_outputs = self.teacher_model(**inputs)

        # 计算标准的监督损失
        loss = outputs_student.loss
        logits = outputs_student.logits
        
        with torch.no_grad():
            teacher_logits = teacher_outputs.logits
        # 如果教师模型和学生模型输出形状不匹配，对学生模型进行padding或对教师模型进行截断
        # print(logits.shape, teacher_logits.shape)
        # print(type(logits), type(teacher_logits))
        # if logits is None or teacher_logits is None:
        kl = 0
        if isinstance(logits, torch.Tensor) and isinstance(teacher_logits, torch.Tensor):
            # 对齐 logits 维度
            if logits.shape[-1] != teacher_logits.shape[-1]:
                # gap = teacher_logits.shape[-1] - logits.shape[-1]
                # if gap > 0:
                # pad_logits = torch.zeros((logits.shape[0],logits.shape[1], gap)).to(logits.device)
                # logits = torch.cat([logits, pad_logits], dim=-1)
                # 截断 teacher_logits 与学生模型保持一致
                teacher_logits = teacher_logits[:, :, :logits.shape[-1]]
                labels = inputs['labels']
                kl = compute_rkl(logits, teacher_logits, labels, padding_id=-100, temp=2.0)
    
        if self.if_use_entropy:
            loss_total = 0.5 * kl + 0.5 * loss
        else:
            loss_total = kl
        return (loss_total, outputs_student) if return_outputs else loss_total


## 2. 加载微调后的 Teacher 模型

In [5]:
max_seq_length = 2048
dtype = None 
load_in_4bit = True
teacher, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/input/qwen-teacher-finetune/qwen_teacher_finetune",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# FastLanguageModel.for_inference(teacher)

==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33+5d4b92a5.d20251029. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


## 3. 加载 Student 模型

In [6]:
# 学生模型
student, _ = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

student = FastLanguageModel.get_peft_model(
    student,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none", # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False, # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


student.print_trainable_parameters()

==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33+5d4b92a5.d20251029. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


trainable params: 10,092,544 || all params: 606,142,464 || trainable%: 1.6650


In [7]:
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split



alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction: 
You are an expert of film comment analysis. Analyze the given text from an online review and determine the sentiment polarity. Respond with 'positive' or 'negative'.   

### Question: 
{} 

### Response: 
{}
"""
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples): 
    inputs = examples["text"] 
    labels = examples["label"]
    outputs = []

    for input, label in zip(inputs, labels): 
        label_text = "positive" if label == 1 else "negative"
        text = alpaca_prompt.format(input, label_text) + EOS_TOKEN 
        outputs.append(text) 
    return { 
        "text": outputs, 
} 


# 加载数据
train = pd.read_csv("/kaggle/input/corpus-imdb/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/corpus-imdb/testData.tsv", header=0, delimiter="\t", quoting=3)
train, val = train_test_split(train, test_size=.2)
    
train_dict = {'label': train["sentiment"], 'text': train['review']}
val_dict = {'label': val["sentiment"], 'text': val['review']}
test_dict = {"text": test['review']}

train_dataset = datasets.Dataset.from_dict(train_dict)
val_dataset = datasets.Dataset.from_dict(val_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)

tokenized_train = train_dataset.map(formatting_prompts_func, batched = True,)
tokenized_val = val_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

## 4. 设置训练参数

In [8]:
# 训练参数
args = TrainingArguments(
    output_dir='results',
    num_train_epochs=2,
    do_train=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    logging_steps=100,
    save_strategy='epoch',
    save_total_limit=10,
    fp16 = not is_bfloat16_supported(),
    bf16 = is_bfloat16_supported(),
    learning_rate=0.001,
    lr_scheduler_type='constant',
    optim="adamw_torch_fused",
    report_to = "none", 
)

trainer = KDTrainer(
    model=student,
    teacher_model=teacher,
    if_use_entropy=True,
    tokenizer=tokenizer,
    train_dataset=tokenized_train, 
    eval_dataset=tokenized_val,
    dataset_text_field = "text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    packing=False, # Can make training 5x faster for short sequences.
    args=args,
)

# 如果是初次训练resume_from_checkpoint为false，接着checkpoint继续训练，为True
trainer.train(resume_from_checkpoint=False)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/20000 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/5000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 2 | Total steps = 10,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)


Step,Training Loss
100,2.625700
200,2.603100
300,2.553600
400,2.566600
500,2.569100
600,2.526800
700,2.584600
800,2.556700
900,2.603500
1000,2.579700


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=10000, training_loss=2.566461376953125, metrics={'train_runtime': 24049.0376, 'train_samples_per_second': 1.663, 'train_steps_per_second': 0.416, 'total_flos': 5.375419416576e+16, 'train_loss': 2.566461376953125, 'epoch': 2.0})

# 三、模型预测

In [9]:
from tqdm import tqdm
import numpy as np

# 提示模板 进行微调需要补充全部的推理指令
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction: 
You are an expert of film comment analysis. Analyze the given text from an online review and determine the sentiment polarity. Respond with 'positive' or 'negative'.   

### Question: 
{} 

### Response: 
"""


def format_test_prompt(examples):
    inputs = examples["text"]
    outputs = []
    for inp in inputs:
        text = prompt_style.format(inp)
        outputs.append(text)
    return {"text": outputs}

formatted_test = test_dataset.map(format_test_prompt, batched=True)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=max_seq_length,
        add_special_tokens=True,
    )

tokenized_test = formatted_test.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
)
tokenizer.padding_side = "left"


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
student.eval()
student.to("cuda")

generation_config = {
    "max_new_tokens": 10,
    "do_sample": False,
    "temperature": 0.0,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

predictions = []
batch_size = 4
total_batches = (len(tokenized_test) + batch_size - 1) // batch_size

for i in tqdm(range(0, len(tokenized_test), batch_size), total=total_batches, desc="Inference"):
    batch = tokenized_test[i:i+batch_size]
    input_ids = [item for item in batch["input_ids"]]
    attention_mask = [item for item in batch["attention_mask"]]

    from transformers import DataCollatorForSeq2Seq
    collator = DataCollatorForSeq2Seq(tokenizer, padding=True, return_tensors="pt")
    batch_collated = collator([{"input_ids": ids, "attention_mask": mask} 
                               for ids, mask in zip(input_ids, attention_mask)])
    
    input_ids = batch_collated["input_ids"].to("cuda")
    attention_mask = batch_collated["attention_mask"].to("cuda")

    with torch.no_grad():
        outputs = student.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **generation_config
        )

    for j in range(len(outputs)):
        prompt_len = input_ids.shape[1]
        generated = tokenizer.decode(outputs[j][prompt_len:], skip_special_tokens=True).strip().lower()
        if "positive" in generated:
            pred = 1
        elif "negative" in generated:
            pred = 0
        else:
            words = generated.split()
            if words and "pos" in words[0]:
                pred = 1
            else:
                pred = 0
        predictions.append(pred)


test_pred = np.array(predictions)

result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/kaggle/working/decoder_distillation.csv", index=False, quoting=3)

Inference:   0%|          | 0/6250 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:963: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 1024], which does not match the required output shape [4, 1, 1024]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /pytorch/aten/src/ATen/native/Resize.cpp:31.)
  out = torch_matmul(X, W.t(), out = out)
/usr/local/lib/python3.11/dist-packages/unsloth/kernels/utils.py:970: UserWarning: An output with one or more elements was resized since it had shape [1, 4, 1024], which does not match the required output shape [4, 1, 1024]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t b